In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [9]:
data=pd.read_csv('Social_Network_Ads.csv')
data.head()

User ID  Gender  Age  EstimatedSalary  Purchased
0  15624510    Male   19            19000          0
1  15810944    Male   35            20000          0
2  15668575  Female   26            43000          0
3  15603246  Female   27            57000          0
4  15804002    Male   19            76000          0

In [10]:
X=data.iloc[:,2:4].values

In [11]:
Y=data.iloc[:,-1].values

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [24]:
n=np.sqrt(X_train.shape[0])
k=n.astype(int)
k

np.int64(17)

In [25]:

from sklearn.neighbors import KNeighborsClassifier
classifier=KNeighborsClassifier(n_neighbors=k)
classifier.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=np.int64(17))

In [26]:
y_pred=classifier.predict(X_test)


In [27]:
y_pred.shape

(80,)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.95

In [32]:
def predict_output():
  age=int(input("Enter the age "))
  salary=int(input("Enter the Salary "))
  X_new=np.array([[age],[salary]]).reshape(1,2)
  X_new=sc.transform(X_new)

  if classifier.predict(X_new)[0]==0:
    return "Will not purchase"
  else:
    return "Will purchase"

In [33]:
predict_output()

Enter the age 20
Enter the Salary 20


'Will not purchase'

In [35]:
import pickle

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(sc, f)

# Save the classifier
with open('model.pkl', 'wb') as f:
    pickle.dump(classifier, f)


In [36]:
# Load them when needed
with open('scaler.pkl', 'rb') as f:
    sc = pickle.load(f)

with open('model.pkl', 'rb') as f:
    classifier = pickle.load(f)


In [37]:
import numpy as np
import pickle

# Load model and scaler
with open('model.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    sc = pickle.load(f)

def predict_output():
    age = int(input("Enter the age: "))
    salary = int(input("Enter the salary: "))

    # Format and scale input
    X_new = np.array([[age, salary]])
    X_new_scaled = sc.transform(X_new)

    # Predict
    prediction = classifier.predict(X_new_scaled)[0]

    return "Will purchase" if prediction == 1 else "Will not purchase"

# Example usage
print(predict_output())


Enter the age: 20
Enter the salary: 20
Will not purchase


In [39]:
print(predict_output())  # Will ask for input

# OR test directly:
test_data = sc.transform([[30, 87000]])


Enter the age: 20
Enter the salary: 20
Will not purchase


In [40]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import pickle

# Load model and scaler
with open('model.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    sc = pickle.load(f)

# Predict function
def predict():
    try:
        age = int(entry_age.get())
        salary = int(entry_salary.get())

        X_new = np.array([[age, salary]])
        X_scaled = sc.transform(X_new)

        result = classifier.predict(X_scaled)[0]

        if result == 1:
            messagebox.showinfo("Prediction", "✅ Will Purchase")
        else:
            messagebox.showinfo("Prediction", "❌ Will Not Purchase")
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter valid numbers for age and salary.")

# GUI Window
root = tk.Tk()
root.title("Purchase Predictor")
root.geometry("300x200")

# Age input
tk.Label(root, text="Enter Age:").pack(pady=5)
entry_age = tk.Entry(root)
entry_age.pack()

# Salary input
tk.Label(root, text="Enter Salary:").pack(pady=5)
entry_salary = tk.Entry(root)
entry_salary.pack()

# Predict button
tk.Button(root, text="Predict", command=predict).pack(pady=20)

root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [43]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00


In [44]:
import gradio as gr
import numpy as np
import pickle

# Load model and scaler
with open('model.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    sc = pickle.load(f)

# Define prediction function
def predict_purchase(age, salary):
    try:
        X = np.array([[int(age), int(salary)]])
        X_scaled = sc.transform(X)
        result = classifier.predict(X_scaled)[0]
        return "✅ Will Purchase" if result == 1 else "❌ Will Not Purchase"
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
iface = gr.Interface(
    fn=predict_purchase,
    inputs=[
        gr.Number(label="Age"),
        gr.Number(label="Salary")
    ],
    outputs=gr.Text(label="Prediction"),
    title="🧠 Purchase Predictor",
    description="Enter age and salary to check if the user is likely to purchase."
)

# Launch app
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://abc297110dc40fa585.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [45]:
iface.launch(share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://abc297110dc40fa585.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
